<a href="https://colab.research.google.com/github/quentinshin34/DS3001-final/blob/main/thyroid_cancer_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code for DS3001 Final Project